# VAR Fairness Audit: LLM Analysis

**DS 112 Final Project**

This notebook uses Google Gemini to analyze VAR incident descriptions.

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn plotly scikit-learn scipy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)

## Setup Google Gemini API

First, let's set up the Google Gemini API for analyzing VAR incident descriptions.

In [ ]:
# Install the Google Generative AI library
!pip install google-generativeai

In [ ]:
# Import necessary libraries
import google.generativeai as genai
import time

# Configure Gemini API (you'll need to provide your API key)
try:
    from google.colab import userdata
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("API configured successfully!")
except:
    print("To use Google Gemini API, add your API key to Colab secrets.")
    print("1. Get an API key from https://ai.google.dev/")
    print("2. Go to Runtime > Manage Secrets and add GEMINI_API_KEY")

## Load Dataset

Let's load the VAR dataset and prepare the incident descriptions for analysis.

In [ ]:
# Load the combined dataset
df = pd.read_csv('var_combined.csv')

# Display the first few incident descriptions
print("Sample VAR Incident Descriptions:")
df[['Description', 'IncidentType']].head()

## Analyze Incident Descriptions

Let's use Google Gemini to analyze VAR incident descriptions for potential bias.

In [ ]:
# Function to analyze an incident description
def analyze_incident(incident_text, decision_type):
    # Configure the model
    model = genai.GenerativeModel('gemini-pro')
    
    # Create the prompt
    prompt = f"""Analyze this soccer/football VAR (Video Assistant Referee) incident description objectively:
    
    Incident: {incident_text}
    Official Decision: {decision_type}
    
    Based solely on this description, please evaluate:
    1. Was this decision justified based on the rules of soccer?
    2. Rate the controversy level of this decision (1-10 scale)
    3. Is there any potential for bias in this decision?
    
    Provide your analysis in a structured format with clear reasoning."""
    
    # Generate response
    response = model.generate_content(prompt)
    return response.text

# Analyze a sample incident
sample_idx = 0  # Change this to analyze different incidents
sample_incident = df.iloc[sample_idx]
print(f"Analyzing incident: {sample_incident['Description']}")
print(f"Decision: {sample_incident['IncidentType']}\n")

# Call the analysis function
try:
    analysis = analyze_incident(sample_incident['Description'], sample_incident['IncidentType'])
    print("Gemini Analysis:")
    print(analysis)
except Exception as e:
    print(f"Error: {e}")
    print("Make sure you've added your API key to Colab secrets.")

## Batch Analysis

Let's analyze multiple incidents and compile the results.

In [ ]:
# Function to analyze multiple incidents
def batch_analyze(incidents_df, n_samples=5):
    # Sample incidents to analyze
    if len(incidents_df) > n_samples:
        samples = incidents_df.sample(n_samples, random_state=42)
    else:
        samples = incidents_df
    
    # Collect results
    results = []
    
    # Analyze each sample
    for i, incident in samples.iterrows():
        try:
            print(f"Analyzing incident {i+1}/{len(samples)}...")
            analysis = analyze_incident(incident['Description'], incident['IncidentType'])
            
            # Add to results
            results.append({
                'IncidentID': i,
                'description': incident['Description'],
                'Decision': incident['IncidentType'],
                'analysis': analysis
            })
            
            # Sleep to avoid rate limiting
            time.sleep(1)
            
        except Exception as e:
            print(f"Error analyzing incident {i}: {e}")
    
    # Convert to DataFrame
    return pd.DataFrame(results)

# Uncomment to run batch analysis
# Set n_samples to a small number for testing
# analysis_results = batch_analyze(df, n_samples=3)
# analysis_results